In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

In [3]:
%matplotlib inline
from sklearn.datasets import make_blobs

Step One

Importing Severe Weather Test Data

In [4]:
# importing test weather data
weather_df = pd.read_csv("SevereWeatherDetails_appended2.csv", low_memory=False)

In [5]:
weather_df.head()

,EVENT_ID,STATE,STATE_FIPS,YEAR,MONTH_NAME,EVENT_TYPE,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,MAGNITUDE,CATEGORY,TOR_F_SCALE,TOR_LENGTH,TOR_WIDTH,BEGIN_LAT,BEGIN_LON
0,10096222,OKLAHOMA,40,1950,April,Tornado,0,0,0,0,250K,0,0.0,0.0,F3,3.4,400.0,35.12,-99.20
1,10120412,TEXAS,48,1950,April,Tornado,0,0,0,0,25K,0,0.0,0.0,F1,11.5,200.0,31.90,-98.60
2,10104927,PENNSYLVANIA,42,1950,July,Tornado,2,0,0,0,25K,0,0.0,0.0,F2,12.9,33.0,40.58,-75.70
3,10104928,PENNSYLVANIA,42,1950,July,Tornado,0,0,0,0,2.5K,0,0.0,0.0,F2,0.0,13.0,40.60,-76.75
4,10104929,PENNSYLVANIA,42,1950,July,Tornado,0,0,0,0,2.5K,0,0.0,0.0,F0,0.0,33.0,41.63,-79.68


In [6]:
# change month to a number count

weather_df['month'] = pd.to_datetime(weather_df.MONTH_NAME, format='%B').dt.month

In [7]:
# dropping columns not needed
weather_df = weather_df.drop(columns=['INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DEATHS_INDIRECT',
                                     'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'MAGNITUDE', 'TOR_F_SCALE', 'TOR_LENGTH',
                                     'TOR_WIDTH', 'BEGIN_LAT', 'BEGIN_LON','CATEGORY', 'STATE', 'MONTH_NAME'])

In [8]:
weather_df.head()

,EVENT_ID,STATE_FIPS,YEAR,EVENT_TYPE,month
0,10096222,40,1950,Tornado,4
1,10120412,48,1950,Tornado,4
2,10104927,42,1950,Tornado,7
3,10104928,42,1950,Tornado,7
4,10104929,42,1950,Tornado,7


In [9]:
weather_df = weather_df.drop(weather_df[weather_df.YEAR > 2000].index)

In [10]:
# displaying weather df
weather_df.head()

,EVENT_ID,STATE_FIPS,YEAR,EVENT_TYPE,month
0,10096222,40,1950,Tornado,4
1,10120412,48,1950,Tornado,4
2,10104927,42,1950,Tornado,7
3,10104928,42,1950,Tornado,7
4,10104929,42,1950,Tornado,7


In [11]:
# adding together events by groups
group_groups = weather_df.groupby(['YEAR', "EVENT_TYPE", "STATE_FIPS"], as_index=False).count()
print(group_groups.head(10))
print(group_groups.tail(10))

   YEAR EVENT_TYPE  STATE_FIPS  EVENT_ID  month
0  1950    Tornado           1         2      2
1  1950    Tornado           5        13     13
2  1950    Tornado           8         3      3
3  1950    Tornado           9         2      2
4  1950    Tornado          12         6      6
5  1950    Tornado          13         4      4
6  1950    Tornado          17        11     11
7  1950    Tornado          18         3      3
8  1950    Tornado          19         4      4
9  1950    Tornado          20        33     33
      YEAR      EVENT_TYPE  STATE_FIPS  EVENT_ID  month
9766  2000  Winter Weather          34         6      6
9767  2000  Winter Weather          36        83     83
9768  2000  Winter Weather          37        56     56
9769  2000  Winter Weather          40        11     11
9770  2000  Winter Weather          42        10     10
9771  2000  Winter Weather          44         2      2
9772  2000  Winter Weather          45        31     31
9773  2000  Winter Weath

In [12]:
# rename 'EVENT_ID' column as 'Count' and 'YEAR' as 'Year'
group_groups = group_groups.rename(columns={'EVENT_ID': 'Count', 'YEAR': 'Year'})
group_groups.head()

,Year,EVENT_TYPE,STATE_FIPS,Count,month
0,1950,Tornado,1,2,2
1,1950,Tornado,5,13,13
2,1950,Tornado,8,3,3
3,1950,Tornado,9,2,2
4,1950,Tornado,12,6,6


In [13]:
# Co2 Below This

In [14]:
# importing test co2 data 
carbon_df = pd.read_csv("co2_byYear.csv", low_memory=False)

In [15]:
# display co2 data
carbon_df

,year,co2,co2_per_capita,co2_per_unit_energy,coal_co2,cement_co2,flaring_co2,gas_co2,oil_co2,other_industry_co2,...,ghg_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp,primary_energy_consumption,energy_per_capita,energy_per_gdp
0,1950,23193.336,943.286,0.000,15959.768,272.887,223.550,1110.931,5611.231,9.976,...,0.000,0.00,0.000,0.00,0.000,7.570713e+09,1.585967e+13,0.000,0.000,0.000
1,1951,24765.194,1214.978,0.000,16981.583,309.265,243.305,1306.674,5907.881,11.002,...,0.000,0.00,0.000,0.00,0.000,7.712655e+09,7.992201e+12,0.000,0.000,0.000
2,1952,25203.424,834.737,0.000,16927.201,331.430,259.877,1405.429,6263.420,10.710,...,0.000,0.00,0.000,0.00,0.000,7.852545e+09,8.401431e+12,0.000,0.000,0.000
3,1953,25929.893,880.578,0.000,17104.163,368.454,248.979,1498.933,6689.993,12.910,...,0.000,0.00,0.000,0.00,0.000,7.992062e+09,8.884521e+12,0.000,0.000,0.000
4,1954,26744.552,1378.627,0.000,17319.801,400.182,236.584,1588.220,7182.609,11.442,...,0.000,0.00,0.000,0.00,0.000,8.133217e+09,9.099744e+12,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,2016,124705.642,1156.831,44.924,48325.275,4990.527,1438.122,27011.160,41892.058,1048.509,...,1415.378,17424.08,345.519,6291.93,105.005,2.239131e+10,2.147076e+14,366170.181,5562418.068,208.589
67,2017,126304.716,1156.251,18.863,48587.789,5067.569,1576.265,27603.881,42412.614,1056.584,...,0.000,0.00,0.000,0.00,0.000,2.264281e+10,2.211738e+14,366923.317,3678183.705,0.000
68,2018,128452.538,1152.783,18.444,49368.748,5286.347,1531.205,28781.873,42422.168,1062.207,...,0.000,0.00,0.000,0.00,0.000,2.289184e+10,2.276454e+14,376507.961,3698476.648,0.000
69,2019,128422.887,1165.000,18.142,48731.161,5408.677,1611.674,29061.768,42547.578,1062.016,...,0.000,0.00,0.000,0.00,0.000,2.313895e+10,0.000000e+00,380148.487,3659567.475,0.000


In [16]:
# rename columns for merging purposes
carbon_df = carbon_df.rename(columns={"year": "Year"})
carbon_df

,Year,co2,co2_per_capita,co2_per_unit_energy,coal_co2,cement_co2,flaring_co2,gas_co2,oil_co2,other_industry_co2,...,ghg_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp,primary_energy_consumption,energy_per_capita,energy_per_gdp
0,1950,23193.336,943.286,0.000,15959.768,272.887,223.550,1110.931,5611.231,9.976,...,0.000,0.00,0.000,0.00,0.000,7.570713e+09,1.585967e+13,0.000,0.000,0.000
1,1951,24765.194,1214.978,0.000,16981.583,309.265,243.305,1306.674,5907.881,11.002,...,0.000,0.00,0.000,0.00,0.000,7.712655e+09,7.992201e+12,0.000,0.000,0.000
2,1952,25203.424,834.737,0.000,16927.201,331.430,259.877,1405.429,6263.420,10.710,...,0.000,0.00,0.000,0.00,0.000,7.852545e+09,8.401431e+12,0.000,0.000,0.000
3,1953,25929.893,880.578,0.000,17104.163,368.454,248.979,1498.933,6689.993,12.910,...,0.000,0.00,0.000,0.00,0.000,7.992062e+09,8.884521e+12,0.000,0.000,0.000
4,1954,26744.552,1378.627,0.000,17319.801,400.182,236.584,1588.220,7182.609,11.442,...,0.000,0.00,0.000,0.00,0.000,8.133217e+09,9.099744e+12,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,2016,124705.642,1156.831,44.924,48325.275,4990.527,1438.122,27011.160,41892.058,1048.509,...,1415.378,17424.08,345.519,6291.93,105.005,2.239131e+10,2.147076e+14,366170.181,5562418.068,208.589
67,2017,126304.716,1156.251,18.863,48587.789,5067.569,1576.265,27603.881,42412.614,1056.584,...,0.000,0.00,0.000,0.00,0.000,2.264281e+10,2.211738e+14,366923.317,3678183.705,0.000
68,2018,128452.538,1152.783,18.444,49368.748,5286.347,1531.205,28781.873,42422.168,1062.207,...,0.000,0.00,0.000,0.00,0.000,2.289184e+10,2.276454e+14,376507.961,3698476.648,0.000
69,2019,128422.887,1165.000,18.142,48731.161,5408.677,1611.674,29061.768,42547.578,1062.016,...,0.000,0.00,0.000,0.00,0.000,2.313895e+10,0.000000e+00,380148.487,3659567.475,0.000


In [17]:
# dropping columns not needed
carbon_df = carbon_df.drop(columns=['co2_per_unit_energy', 'coal_co2', 'cement_co2', 'flaring_co2', 'gas_co2',
                                         'oil_co2', 'other_industry_co2', 'ghg_per_capita', 'methane', 'methane_per_capita',
                                         'nitrous_oxide', 'nitrous_oxide_per_capita', 'population', 'gdp', 'primary_energy_consumption',
                                         'energy_per_capita', 'energy_per_gdp', 'total_ghg'])

In [18]:
carbon_df = carbon_df.drop(carbon_df[carbon_df.Year > 2000].index)

In [19]:
# show new dataframe
carbon_df

,Year,co2,co2_per_capita
0,1950,23193.336,943.286
1,1951,24765.194,1214.978
2,1952,25203.424,834.737
3,1953,25929.893,880.578
4,1954,26744.552,1378.627
5,1955,29264.525,1214.075
6,1956,31190.093,1417.809
7,1957,32296.584,1096.454
8,1958,33117.274,996.436
9,1959,34633.663,1119.316


In [20]:
# Merge attempt one
combo_df = group_groups.merge(carbon_df, left_on='Year', right_on='Year')

In [21]:
# checking merge
combo_df

,Year,EVENT_TYPE,STATE_FIPS,Count,month,co2,co2_per_capita
0,1950,Tornado,1,2,2,23193.336,943.286
1,1950,Tornado,5,13,13,23193.336,943.286
2,1950,Tornado,8,3,3,23193.336,943.286
3,1950,Tornado,9,2,2,23193.336,943.286
4,1950,Tornado,12,6,6,23193.336,943.286
...,...,...,...,...,...,...,...
9771,2000,Winter Weather,44,2,2,92998.026,1174.210
9772,2000,Winter Weather,45,31,31,92998.026,1174.210
9773,2000,Winter Weather,50,113,113,92998.026,1174.210
9774,2000,Winter Weather,51,49,49,92998.026,1174.210


In [22]:
# Generate our categorical variable list
combo_cat = combo_df.dtypes[combo_df.dtypes == "object"].index.tolist()

In [23]:
# Check the number of unique values in each column
combo_df[combo_cat].nunique()

EVENT_TYPE    54
dtype: int64

In [24]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(combo_df[combo_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(combo_cat)
encode_df.head()

,EVENT_TYPE_Astronomical Low Tide,EVENT_TYPE_Avalanche,EVENT_TYPE_Blizzard,EVENT_TYPE_Coastal Flood,EVENT_TYPE_Cold/Wind Chill,EVENT_TYPE_Debris Flow,EVENT_TYPE_Dense Fog,EVENT_TYPE_Drought,EVENT_TYPE_Dust Devil,EVENT_TYPE_Dust Storm,...,EVENT_TYPE_TORNADO/WATERSPOUT,"EVENT_TYPE_TORNADOES, TSTM WIND, HAIL",EVENT_TYPE_Thunderstorm Wind,EVENT_TYPE_Tornado,EVENT_TYPE_Tropical Storm,EVENT_TYPE_Volcanic Ash,EVENT_TYPE_Waterspout,EVENT_TYPE_Wildfire,EVENT_TYPE_Winter Storm,EVENT_TYPE_Winter Weather
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
# Merge one-hot encoded features and drop the originals
combo_df = combo_df.merge(encode_df,left_index=True, right_index=True)
combo_df = combo_df.drop(combo_cat,1)
combo_df.head()

,Year,STATE_FIPS,Count,month,co2,co2_per_capita,EVENT_TYPE_Astronomical Low Tide,EVENT_TYPE_Avalanche,EVENT_TYPE_Blizzard,EVENT_TYPE_Coastal Flood,...,EVENT_TYPE_TORNADO/WATERSPOUT,"EVENT_TYPE_TORNADOES, TSTM WIND, HAIL",EVENT_TYPE_Thunderstorm Wind,EVENT_TYPE_Tornado,EVENT_TYPE_Tropical Storm,EVENT_TYPE_Volcanic Ash,EVENT_TYPE_Waterspout,EVENT_TYPE_Wildfire,EVENT_TYPE_Winter Storm,EVENT_TYPE_Winter Weather
0,1950,1,2,2,23193.336,943.286,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1950,5,13,13,23193.336,943.286,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1950,8,3,3,23193.336,943.286,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1950,9,2,2,23193.336,943.286,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1950,12,6,6,23193.336,943.286,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
combo_df.dtypes

Year                                           int64
STATE_FIPS                                     int64
Count                                          int64
month                                          int64
co2                                          float64
co2_per_capita                               float64
EVENT_TYPE_Astronomical Low Tide             float64
EVENT_TYPE_Avalanche                         float64
EVENT_TYPE_Blizzard                          float64
EVENT_TYPE_Coastal Flood                     float64
EVENT_TYPE_Cold/Wind Chill                   float64
EVENT_TYPE_Debris Flow                       float64
EVENT_TYPE_Dense Fog                         float64
EVENT_TYPE_Drought                           float64
EVENT_TYPE_Dust Devil                        float64
EVENT_TYPE_Dust Storm                        float64
EVENT_TYPE_Excessive Heat                    float64
EVENT_TYPE_Extreme Cold/Wind Chill           float64
EVENT_TYPE_Flash Flood                       f

In [27]:
# Split our preprocessed data into our features and target arrays
y = combo_df["Count"].values
X = combo_df.drop(["Count","co2_per_capita", "STATE_FIPS"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [28]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [54]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  5
hidden_nodes_layer2 = 2

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 5)                 290       
                                                                 
 dense_24 (Dense)            (None, 2)                 12        
                                                                 
 dense_25 (Dense)            (None, 1)                 3         
                                                                 
Total params: 305
Trainable params: 305
Non-trainable params: 0
_________________________________________________________________


In [55]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
197/230 [========================>.....] - ETA: 0s - loss: -117.8160 - accuracy: 0.0898
Epoch 00001: saving model to checkpoints\weights.01.hdf5
230/230 [==============================] - 1s 1ms/step - loss: -149.7246 - accuracy: 0.0896
Epoch 2/100
205/230 [=========================>....] - ETA: 0s - loss: -887.8462 - accuracy: 0.0883
Epoch 00002: saving model to checkpoints\weights.02.hdf5
230/230 [==============================] - 0s 1ms/step - loss: -1003.5628 - accuracy: 0.0896
Epoch 3/100
195/230 [========================>.....] - ETA: 0s - loss: -3185.9915 - accuracy: 0.0902
Epoch 00003: saving model to checkpoints\weights.03.hdf5
230/230 [==============================] - 0s 1ms/step - loss: -3495.6809 - accuracy: 0.0896
Epoch 4/100
202/230 [=========================>....] - ETA: 0s - loss: -8337.4805 - accuracy: 0.0897
Epoch 00004: saving model to checkpoints\weights.04.hdf5
230/230 [==============================] - 0s 1ms/step - loss: -8403.8242 - accuracy: 0.0896

196/230 [========================>.....] - ETA: 0s - loss: -2250730.0000 - accuracy: 0.0894
Epoch 00033: saving model to checkpoints\weights.33.hdf5
230/230 [==============================] - 0s 1ms/step - loss: -2254186.0000 - accuracy: 0.0896
Epoch 34/100
203/230 [=========================>....] - ETA: 0s - loss: -2440930.0000 - accuracy: 0.0897
Epoch 00034: saving model to checkpoints\weights.34.hdf5
230/230 [==============================] - 0s 1ms/step - loss: -2423180.7500 - accuracy: 0.0896
Epoch 35/100
204/230 [=========================>....] - ETA: 0s - loss: -2634423.7500 - accuracy: 0.0890
Epoch 00035: saving model to checkpoints\weights.35.hdf5
230/230 [==============================] - 0s 1ms/step - loss: -2600396.5000 - accuracy: 0.0896
Epoch 36/100
202/230 [=========================>....] - ETA: 0s - loss: -2802346.7500 - accuracy: 0.0857
Epoch 00036: saving model to checkpoints\weights.36.hdf5
230/230 [==============================] - 0s 1ms/step - loss: -2786025.2500 

Epoch 65/100
204/230 [=========================>....] - ETA: 0s - loss: -11695134.0000 - accuracy: 0.0898
Epoch 00065: saving model to checkpoints\weights.65.hdf5
230/230 [==============================] - 0s 1ms/step - loss: -12037674.0000 - accuracy: 0.0896
Epoch 66/100
205/230 [=========================>....] - ETA: 0s - loss: -12467778.0000 - accuracy: 0.0892
Epoch 00066: saving model to checkpoints\weights.66.hdf5
230/230 [==============================] - 0s 1ms/step - loss: -12503763.0000 - accuracy: 0.0896
Epoch 67/100
203/230 [=========================>....] - ETA: 0s - loss: -13054890.0000 - accuracy: 0.0894
Epoch 00067: saving model to checkpoints\weights.67.hdf5
230/230 [==============================] - 0s 1ms/step - loss: -12977531.0000 - accuracy: 0.0896
Epoch 68/100
207/230 [==========================>...] - ETA: 0s - loss: -13287799.0000 - accuracy: 0.0891
Epoch 00068: saving model to checkpoints\weights.68.hdf5
230/230 [==============================] - 0s 1ms/step - 

In [40]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

77/77 - 0s - loss: -4.4388e+08 - accuracy: 0.0941 - 275ms/epoch - 4ms/step
Loss: -443880224.0, Accuracy: 0.09410802274942398


In [33]:
# Export our model to HDF5 file
nn_new.save("trained_attrition.h5")

In [34]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('trained_attrition.h5')

In [35]:
# Evaluate the completed model using the test data
model_loss, model_accuracy = nn_imported.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

77/77 - 0s - loss: -2.5102e+07 - accuracy: 0.0941 - 211ms/epoch - 3ms/step
Loss: -25102002.0, Accuracy: 0.09410802274942398


In [36]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [37]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
230/230 [==============================] - 1s 1ms/step - loss: -402387107840.0000 - accuracy: 0.0896
Epoch 2/100
230/230 [==============================] - 0s 1ms/step - loss: -413592518656.0000 - accuracy: 0.0896
Epoch 3/100
230/230 [==============================] - 0s 1ms/step - loss: -425071280128.0000 - accuracy: 0.0896
Epoch 4/100
230/230 [==============================] - 0s 1ms/step - loss: -437003681792.0000 - accuracy: 0.0896
Epoch 5/100
230/230 [==============================] - 0s 1ms/step - loss: -449215135744.0000 - accuracy: 0.0896
Epoch 6/100
230/230 [==============================] - 0s 1ms/step - loss: -461662978048.0000 - accuracy: 0.0896
Epoch 7/100
230/230 [==============================] - 0s 1ms/step - loss: -474453311488.0000 - accuracy: 0.0896
Epoch 8/100
230/230 [==============================] - 0s 1ms/step - loss: -487556874240.0000 - accuracy: 0.0896
Epoch 9/100
230/230 [==============================] - 0s 1ms/step - loss: -500946763776.0000 - 

230/230 [==============================] - ETA: 0s - loss: -1901789708288.0000 - accuracy: 0.0883- ETA: 0s - loss: -1954596651008.0000 - accuracy: 0. - 0s 1ms/step - loss: -1907685326848.0000 - accuracy: 0.0896
Epoch 72/100
230/230 [==============================] - 0s 1ms/step - loss: -1940755185664.0000 - accuracy: 0.0896
Epoch 73/100
230/230 [==============================] - 0s 1ms/step - loss: -1974201352192.0000 - accuracy: 0.0896
Epoch 74/100
230/230 [==============================] - 0s 1ms/step - loss: -2008092770304.0000 - accuracy: 0.0896
Epoch 75/100
230/230 [==============================] - 0s 1ms/step - loss: -2042496548864.0000 - accuracy: 0.0896
Epoch 76/100
230/230 [==============================] - 0s 1ms/step - loss: -2077087760384.0000 - accuracy: 0.0896
Epoch 77/100
230/230 [==============================] - 0s 1ms/step - loss: -2111978078208.0000 - accuracy: 0.0896
Epoch 78/100
230/230 [==============================] - 0s 1ms/step - loss: -2147263709184.0000 - a

In [38]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

77/77 - 0s - loss: -2.8570e+12 - accuracy: 0.0941 - 204ms/epoch - 3ms/step
Loss: -2856964325376.0, Accuracy: 0.09410802274942398
